In [1]:
%load_ext rpy2.ipython

In [24]:
%%R
library(DESeq2)
library(magrittr)
library(SummarizedExperiment)
library(dplyr)

start_time <- Sys.time()

IN_DIR <- "/mnt/home/yuankeji/RanceLab/reticula_new/reticula/data/tcdd/input/"
OUT_DIR <- "/mnt/home/yuankeji/RanceLab/reticula_new/reticula/data/tcdd/output/"

GTEx_DATA_DIR <- IN_DIR
GTEx_DATA_FIL <- "rse_tcdd_data.Rdata"

ensembl2rxns.df <- read.table(paste(IN_DIR,"Ensembl2ReactomeReactions.txt",sep=""),
                              sep="\t")

load(paste(GTEx_DATA_DIR,GTEx_DATA_FIL,sep=""))

In [ ]:
%%R

variances <- apply(assay(final_result), 1, var)

zero_variance_rows <- which(variances == 0)

if (length(zero_variance_rows) > 0) {
    deleted_data <- assay(final_result)[zero_variance_rows, , drop = FALSE]
    save(deleted_data, file=paste0(OUT_DIR, "deleted_data.RData"))
    new_assays <- assay(final_result)[-zero_variance_rows, , drop = FALSE]
    new_row_data <- rowData(final_result)[-zero_variance_rows, , drop = FALSE]

    new_final_result <- SummarizedExperiment(
        assays = SimpleList(counts = new_assays),
        rowData = new_row_data,
        colData = colData(final_result)
    )
    
    final_result <- new_final_result
    
    print(dim(assay(final_result)))
    print(dim(rowData(final_result)))
} else {
    cat("No rows with zero variance found.\n")
}

print(head(assay(final_result)))

[1] 43252   383
[1] 43252    11
                     [,1] [,2] [,3] [,4] [,5] [,6] [,7] [,8] [,9] [,10] [,11]
ENSMUSG00000079800.2    0    0    0    0    0    0    0    0   49    49     0
ENSMUSG00000095092.1    0    0    0    0    0    0    0    0    0     0     0
ENSMUSG00000079192.2    0    0    0    0    0    0    0    0    0     0     0
ENSMUSG00000079794.2    0    0    0    0    0    0    0    0    0     0    50
ENSMUSG00000094799.1    0    0    0    0    0    0    0    0    0     0     0
ENSMUSG00000095250.1    0    0    0    0    0   50    0    0    0     0     0
                     [,12] [,13] [,14] [,15] [,16] [,17] [,18] [,19] [,20]
ENSMUSG00000079800.2    50     0    50    50     0     0     0     0     0
ENSMUSG00000095092.1     0     0     0     0     0     0     0     0     0
ENSMUSG00000079192.2     0     0     0     0     0     0     0     0     0
ENSMUSG00000079794.2     0     0     0     0     0     0     0     0     0
ENSMUSG00000094799.1     0     0     0     0   

ENSMUSG00000095092.1      0      0      0      0      0      0      0      0
ENSMUSG00000079192.2      0      0      0      0      0      0      0      0
ENSMUSG00000079794.2      0      0      0      0      0      0      0      0
ENSMUSG00000094799.1      0      0      0      0      0      0      0      0
ENSMUSG00000095250.1      0      0      0     50     48      0      0      0
                     [,142] [,143] [,144] [,145] [,146] [,147] [,148] [,149]
ENSMUSG00000079800.2      0      0      0      0      0      0      0      0
ENSMUSG00000095092.1      0      0      0      0      0      0      0      0
ENSMUSG00000079192.2      0      0      0      0      0      0      0      0
ENSMUSG00000079794.2      0      0      0      0      0      0      0      0
ENSMUSG00000094799.1      0      0      0      0      0      0      0      0
ENSMUSG00000095250.1      0      0      0      0      0      0      0      0
                     [,150] [,151] [,152] [,153] [,154] [,155] [,156] [,157]

ENSMUSG00000079192.2      0      0      0      0      0      0      0      0
ENSMUSG00000079794.2      0      0      0      0      0      0      0      0
ENSMUSG00000094799.1      0      0      0      0      0      0      0      0
ENSMUSG00000095250.1      0      0      0      0      0      0      0      0
                     [,262] [,263] [,264] [,265] [,266] [,267] [,268] [,269]
ENSMUSG00000079800.2      0      0      0      0      0    100    150     50
ENSMUSG00000095092.1      0      0      0      0      0      0      0      0
ENSMUSG00000079192.2      0      0      0      0      0      0      0      0
ENSMUSG00000079794.2      0      0      0      0      0      0      0      0
ENSMUSG00000094799.1      0      0      0      0      0      0      0      0
ENSMUSG00000095250.1      0      0      0      0      0      0      0      0
                     [,270] [,271] [,272] [,273] [,274] [,275] [,276] [,277]
ENSMUSG00000079800.2      0      0     50      0     50     40     34     49

ENSMUSG00000079794.2      0      0      0      0      0      0      0      0
ENSMUSG00000094799.1      0      0     50     50      0      0      0      0
ENSMUSG00000095250.1     49     50      0      0     50      0     49      0
                     [,382] [,383]
ENSMUSG00000079800.2      0      0
ENSMUSG00000095092.1      0      0
ENSMUSG00000079192.2      0      0
ENSMUSG00000079794.2      0      0
ENSMUSG00000094799.1      0      0
ENSMUSG00000095250.1     48      0


In [ ]:
%%R
library(tibble)

keep_samples <- colData(final_result)$dose %in% c(0.00, 10.00)

final_result <- final_result[, keep_samples]

temp_df <- as_tibble(colData(final_result)) %>%
  filter(!grepl("SRP131784", study))

tcdd_data <- temp_df
final_result <- final_result[, colData(final_result)$external_id %in% temp_df$external_id]

print(dim(final_result))
print(assay(final_result))
print(dim(tcdd_data))
print(tcdd_data)

[1] 43252    90
                          [,1]     [,2]     [,3]     [,4]     [,5]     [,6]
ENSMUSG00000079800.2         0        0        0        0        0       50
ENSMUSG00000095092.1         0        0        0        0        0        0
ENSMUSG00000079192.2         0        0        0        0        0        0
ENSMUSG00000079794.2         0        0        0        0        0        0
ENSMUSG00000094799.1         0        0        0        0        0        0
ENSMUSG00000095250.1         0        0        0        0        0        0
ENSMUSG00000095787.1         0        0        0        0        0        0
ENSMUSG00000096100.1         0        0        0        0        0        0
ENSMUSG00000094054.1         0        0        0        0        0        0
ENSMUSG00000095672.1         0        0        0        0        0        0
ENSMUSG00000079190.3         0       50        0        0      150      150
ENSMUSG00000094514.1        49        0        0       96        0      

ENSMUSG00000103067.1         0        0        0        0        0        0
ENSMUSG00000104226.1       100       50       50      100      150        0
ENSMUSG00000103903.1        50        0        0        0        0       50
ENSMUSG00000102647.1         0        0      100        0        0        0
ENSMUSG00000086235.1       598      500      399      800      650      150
ENSMUSG00000102253.1         0        0        0        0        0        0
ENSMUSG00000103884.1         0        0        0        0        0        0
ENSMUSG00000084353.1      4057     5357     4781     2862     2448     3574
ENSMUSG00000103933.1     31020    29109    46442    34434    34721    30695
ENSMUSG00000076135.1         0        8        0        0        0        0
ENSMUSG00000086195.1        36       68       20        0        0       24
ENSMUSG00000104504.1        31        6        0       12       18       26
ENSMUSG00000066693.2      3448     3942     5114     3616     4694     4299
ENSMUSG00000

ENSMUSG00000104165.1         0        0        0        0        0        0
ENSMUSG00000100868.1        50       50        0        0        0        0
ENSMUSG00000089982.7         0        0        0        0        0        0
ENSMUSG00000025920.19     1547     1878     3114     2520     2405     3553
ENSMUSG00000025939.18    10834    10995    12300    10764    10840     9985
ENSMUSG00000102457.1         0        0        0        0        0        0
ENSMUSG00000079658.9     18194    17088    25529    18559    18636    17940
ENSMUSG00000097744.1         0        0        0        0        0        0
ENSMUSG00000100959.1         0        0      100       49        0        0
ENSMUSG00000025940.6     21675    21923    28179    22316    23280    21717
ENSMUSG00000025779.10     1239     1336     2529     2267     2509     2235
ENSMUSG00000100093.1         0        0       50        0        0        0
ENSMUSG00000091020.3      2844     3042     3237     3246     2149     2149
ENSMUSG00000

ENSMUSG00000101301.1         0        0        0        0        0        0
ENSMUSG00000104221.1         0        0        0        0        0        0
ENSMUSG00000102317.1         0        0        0        0        0        0
ENSMUSG00000102659.1         0        0        0        0        0        0
ENSMUSG00000102747.1         0        0        0        0        0        0
ENSMUSG00000102386.1         0        0        0        0        0        0
ENSMUSG00000033569.17        0        0        0        0        0        0
ENSMUSG00000104516.1         0        0        0        0        0        0
ENSMUSG00000099458.1         0        0       45       43       43        0
ENSMUSG00000100310.1         0        0        0        0        0        0
ENSMUSG00000101033.1      1869     2445     5464     3157     2983     3888
ENSMUSG00000101514.1      5324     5359    14206    10611     9326    13754
ENSMUSG00000101372.1       932     1435      945     1782     1288     1033
ENSMUSG00000

ENSMUSG00000102762.1         0        0        0       50        0        0
ENSMUSG00000098680.1         7        2        3        2        2       12
ENSMUSG00000010453.12    33948    32801    50404    30156    29349    38217
ENSMUSG00000037432.15     1590     1622     1246     1109     1351     1308
ENSMUSG00000001143.13     8646     7832    13080     9712     8308     6883
ENSMUSG00000104316.1       100        0       43       49        0        0
ENSMUSG00000106081.1        29       62       42       31       54      119
ENSMUSG00000037408.10     2984     2392     2745     3346     2436     2976
ENSMUSG00000102845.1        77      160      106       79        2      118
ENSMUSG00000001138.13    11274    11022    14452    12200    11106    13414
ENSMUSG00000067653.12     3711     3335     3857     2107     2136     1716
ENSMUSG00000109510.1      3937     4595     3211     2612     2567     2449
ENSMUSG00000079610.9      3640     4078     4563     3317     3249     2190
ENSMUSG00000

...

ENSMUSG00000026028.12    17540    34715    50556    49990    33306    36964
ENSMUSG00000026027.13    20613    35018    50933    50881    30452    29004
ENSMUSG00000047383.8         0        0        0        0        0        0
ENSMUSG00000099752.1         0        0        0        0        0        0
ENSMUSG00000072295.7         0        0       34        0      100        0
ENSMUSG00000038079.14     2055     5085     4746     4722     3589     4412
ENSMUSG00000097573.2       477      499      768      550      200      250
ENSMUSG00000099563.1         0       50        0       50        0        0
ENSMUSG00000079550.9      3278     5763     6609     5808     2706     3070
ENSMUSG00000101586.1      4383     2732     4059     3403     2157     2504
ENSMUSG00000026024.14    47855    37100    56154    57597    35539    35438
ENSMUSG00000026023.16        0      313      845      696      152      256
ENSMUSG00000099922.1         0        0       49        0       50        0
ENSMUSG00000

ENSMUSG00000069085.5         0        0        0        0        0        0
ENSMUSG00000025963.5         0        0        0        0        0       50
ENSMUSG00000025962.15    10616    12003    19306    19484    12522    12861
ENSMUSG00000025961.1         0       50        0        0       50       50
ENSMUSG00000080817.1         0        0        0        0        0        0
ENSMUSG00000044689.5         0        0        0      100        0        0
ENSMUSG00000098487.1         0        0        1        1        0        0
ENSMUSG00000086836.1         0        4        0       79        0        0
ENSMUSG00000025959.13     3880     7981     7916     7428     6283     7447
ENSMUSG00000086721.2         0      150      100      100       50       50
ENSMUSG00000101895.1         0        0        0        0        0        0
ENSMUSG00000099658.1         0       37      183       50        0        0
ENSMUSG00000025958.14    10237    19282    29859    28766    17004    16382
ENSMUSG00000

ENSMUSG00000101764.1         0        0        0        0        0        0
ENSMUSG00000101496.1      1899     3100     3750     4200     2550     2600
ENSMUSG00000102123.1       100        0        0      100        0        0
ENSMUSG00000039395.8     37119    29525    67829    70533    25110    24424
ENSMUSG00000094655.1         0        0       45        0        0        0
ENSMUSG00000094050.1         0        0        0        0        0        0
ENSMUSG00000093956.1         0        0        0        0        0        0
ENSMUSG00000093843.1         0        0       52       50        0        0
ENSMUSG00000026189.13   144632    86479   107239   104305    42931    44712
ENSMUSG00000026188.11      145      100      100      150      100       50
ENSMUSG00000026187.9      9623    14325    19565    19151    14805    13501
ENSMUSG00000085812.1       494     1622     2488     2696     1320      830
ENSMUSG00000039372.5         0        0        0        0        0        0
ENSMUSG00000

ENSMUSG00000082012.1         0        0        0       39        0        0
ENSMUSG00000100553.6         0        0        0        0        0        0
ENSMUSG00000100534.1         0        0        0        0        0        0
ENSMUSG00000102904.1         0        0        0        0        0        0
ENSMUSG00000026235.14       50      599      100      393       50      449
ENSMUSG00000101849.1         0        0        0        0        0        0
ENSMUSG00000101506.1         0        0        0        0        0        0
ENSMUSG00000102052.1         0        0        0        0        0        0
ENSMUSG00000004872.15        0        0        0        0        0        0
ENSMUSG00000102945.1         0        0        0        0        0        0
ENSMUSG00000032908.9       250

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [ ]:
%%R
dose_counts <- table(tcdd_data$dose)

print(dose_counts)

dose_counts_df <- as.data.frame(dose_counts)

colnames(dose_counts_df) <- c("Dose", "Count")

write.csv(dose_counts_df, paste0(OUT_DIR, "dose_counts10vs0_time_cross.csv"), row.names=FALSE)


 0 10 
68 22 


In [28]:
%%R
combined_gender <- ifelse(tcdd_data$Sex != "", tcdd_data$Sex, tcdd_data$gender)
print(combined_gender)

 [1] "male"   "male"   "male"   "male"   "male"   "male"   "male"   "male"  
 [9] "male"   "male"   "male"   "male"   "male"   "male"   "male"   "male"  
[17] "male"   "male"   "male"   "male"   "male"   "male"   "male"   "male"  
[25] "male"   "male"   "male"   "male"   "male"   "male"   "male"   "male"  
[33] "male"   "male"   "male"   "male"   "male"   "male"   "male"   "male"  
[41] "male"   "male"   "male"   "male"   "male"   "male"   "male"   "male"  
[49] "female" "female" "female" "female" "female" "female" "female" "female"
[57] "female" "female" "female" "female" "female" "female" "female" "female"
[65] "female" "female" "female" "female" "female" "female" "female" "female"
[73] "female" "female" "female" "female" "female" "female" "male"   "male"  
[81] "male"   "male"   "male"   "male"   "male"   "male"   "male"   "male"  
[89] "male"   "male"  


In [29]:
%%R
write.table(tcdd_data$project_id,file=paste(OUT_DIR,"tcdd_project_id10vs0_time_cross.txt",sep=""), row.names = FALSE, col.names = FALSE, sep = "\t")

In [30]:
%%R
write.table(combined_gender,file=paste(OUT_DIR,"tcdd_gender10vs0_time_cross.txt",sep=""), row.names = FALSE, col.names = FALSE, sep = "\t")

In [31]:
%%R
write.table(tcdd_data$external_id,file=paste(OUT_DIR,"tcdd_sample_id10vs0_time_cross.txt",sep=""), row.names = FALSE, col.names = FALSE, sep = "\t")

In [32]:
%%R
write.table(tcdd_data$dose,file=paste(OUT_DIR,"tcdd_dose10vs0_time_cross.txt",sep=""), row.names = FALSE, col.names = FALSE, sep = "\t")

In [33]:
%%R
saveRDS(tcdd_data$dose,file=paste(OUT_DIR,"tcdd_dose_detail_vec10vs0_time_cross.Rds",sep=""))

In [34]:
%%R
saveRDS(tcdd_data$external_id,file=paste(OUT_DIR,"tcdd_sample_detail_vec10vs0_time_cross.Rds",sep=""))

In [35]:
%%R
tcdd.df <- final_result %>% SummarizedExperiment::assay() %>% as.data.frame()

In [36]:
%%R
colnames(tcdd.df) <- colData(final_result)$external_id

In [37]:
%%R
ensembl_wo_ids <- gsub("\\.[0-9]+","",rownames(tcdd.df))
# print(ensembl_wo_ids)

In [38]:
%%R
deleted_ensembl_wo_ids <- gsub("\\.[0-9]+","",rownames(deleted_data))
print(length(deleted_ensembl_wo_ids))

[1] 12169


In [39]:
%%R
rownames(tcdd.df) <- ensembl_wo_ids
# print(rownames(tcdd.df))

In [40]:
%%R
rownames(deleted_data) <- deleted_ensembl_wo_ids
print(rownames(deleted_data))

    [1] "ENSMUSG00000096776" "ENSMUSG00000096244" "ENSMUSG00000095450"
    [4] "ENSMUSG00000089163" "ENSMUSG00000099278" "ENSMUSG00000094121"
    [7] "ENSMUSG00000079764" "ENSMUSG00000079774" "ENSMUSG00000094741"
   [10] "ENSMUSG00000074720" "ENSMUSG00000094337" "ENSMUSG00000095570"
   [13] "ENSMUSG00000098647" "ENSMUSG00000094431" "ENSMUSG00000095247"
   [16] "ENSMUSG00000095585" "ENSMUSG00000096873" "ENSMUSG00000095755"
   [19] "ENSMUSG00000095207" "ENSMUSG00000093828" "ENSMUSG00000095623"
   [22] "ENSMUSG00000095666" "ENSMUSG00000096680" "ENSMUSG00000096236"
   [25] "ENSMUSG00000096756" "ENSMUSG00000094722" "ENSMUSG00000095728"
   [28] "ENSMUSG00000094836" "ENSMUSG00000094383" "ENSMUSG00000094474"
   [31] "ENSMUSG00000096550" "ENSMUSG00000094172" "ENSMUSG00000091585"
   [34] "ENSMUSG00000095763" "ENSMUSG00000095523" "ENSMUSG00000095475"
   [37] "ENSMUSG00000095019" "ENSMUSG00000102693" "ENSMUSG00000064842"
   [40] "ENSMUSG00000102851" "ENSMUSG00000103377" "ENSMUSG00000102348"
   [43

  [346] "ENSMUSG00000104104" "ENSMUSG00000087883" "ENSMUSG00000099892"
  [349] "ENSMUSG00000100977" "ENSMUSG00000077039" "ENSMUSG00000100684"
  [352] "ENSMUSG00000099770" "ENSMUSG00000088510" "ENSMUSG00000101890"
  [355] "ENSMUSG00000087908" "ENSMUSG00000088594" "ENSMUSG00000100955"
  [358] "ENSMUSG00000099469" "ENSMUSG00000101297" "ENSMUSG00000100059"
  [361] "ENSMUSG00000088161" "ENSMUSG00000100935" "ENSMUSG00000103991"
  [364] "ENSMUSG00000084704" "ENSMUSG00000103060" "ENSMUSG00000089866"
  [367] "ENSMUSG00000089206" "ENSMUSG00000084228" "ENSMUSG00000044594"
  [370] "ENSMUSG00000058017" "ENSMUSG00000080729" "ENSMUSG00000073602"
  [373] "ENSMUSG00000073601" "ENSMUSG00000087210" "ENSMUSG00000089278"
  [376] "ENSMUSG00000077237" "ENSMUSG00000099484" "ENSMUSG00000104478"
  [379] "ENSMUSG00000101231" "ENSMUSG00000100884" "ENSMUSG00000089037"
  [382] "ENSMUSG00000084741" "ENSMUSG00000100290" "ENSMUSG00000100758"
  [385] "ENSMUSG00000102728" "ENSMUSG00000088691" "ENSMUSG00000099884"
  [388

  [691] "ENSMUSG00000077749" "ENSMUSG00000111753" "ENSMUSG00000088423"
  [694] "ENSMUSG00000044499" "ENSMUSG00000103299" "ENSMUSG00000111549"
  [697] "ENSMUSG00000096080" "ENSMUSG00000077576" "ENSMUSG00000111234"
  [700] "ENSMUSG00000088925" "ENSMUSG00000111048" "ENSMUSG00000111208"
  [703] "ENSMUSG00000110782" "ENSMUSG00000094521" "ENSMUSG00000094311"
  [706] "ENSMUSG00000111183" "ENSMUSG00000019834" "ENSMUSG00000111098"
  [709] "ENSMUSG00000110889" "ENSMUSG00000046922" "ENSMUSG00000111576"
  [712] "ENSMUSG00000094663" "ENSMUSG00000111315" "ENSMUSG00000093298"
  [715] "ENSMUSG00000087891" "ENSMUSG00000099893" "ENSMUSG00000098642"
  [718] "ENSMUSG00000076282" "ENSMUSG00000110891" "ENSMUSG00000077082"
  [721] "ENSMUSG00000098418" "ENSMUSG00000077591" "ENSMUSG00000098993"
  [724] "ENSMUSG00000098409" "ENSMUSG00000111182" "ENSMUSG00000094681"
  [727] "ENSMUSG00000065718" "ENSMUSG00000111499" "ENSMUSG00000111419"
  [730] "ENSMUSG00000110843" "ENSMUSG00000111719" "ENSMUSG00000112062"
  [733

 [1036] "ENSMUSG00000107400" "ENSMUSG00000094734" "ENSMUSG00000108114"
 [1039] "ENSMUSG00000058513" "ENSMUSG00000088147" "ENSMUSG00000096858"
 [1042] "ENSMUSG00000094702" "ENSMUSG00000071065" "ENSMUSG00000050478"
 [1045] "ENSMUSG00000069421" "ENSMUSG00000063173" "ENSMUSG00000049052"
 [1048] "ENSMUSG00000052818" "ENSMUSG00000061961" "ENSMUSG00000107399"
 [1051] "ENSMUSG00000058071" "ENSMUSG00000094295" "ENSMUSG00000095608"
 [1054] "ENSMUSG00000107423" "ENSMUSG00000048745" "ENSMUSG00000044897"
 [1057] "ENSMUSG00000108253" "ENSMUSG00000095804" "ENSMUSG00000112688"
 [1060] "ENSMUSG00000059862" "ENSMUSG00000045559" "ENSMUSG00000088352"
 [1063] "ENSMUSG00000085559" "ENSMUSG00000085512" "ENSMUSG00000064632"
 [1066] "ENSMUSG00000077304" "ENSMUSG00000088688" "ENSMUSG00000080833"
 [1069] "ENSMUSG00000086808" "ENSMUSG00000082897" "ENSMUSG00000098853"
 [1072] "ENSMUSG00000084318" "ENSMUSG00000089247" "ENSMUSG00000104137"
 [1075] "ENSMUSG00000088861" "ENSMUSG00000064766" "ENSMUSG00000087990"
 [1078

 [1381] "ENSMUSG00000086617" "ENSMUSG00000084200" "ENSMUSG00000064788"
 [1384] "ENSMUSG00000000125" "ENSMUSG00000080694" "ENSMUSG00000089479"
 [1387] "ENSMUSG00000086089" "ENSMUSG00000087184" "ENSMUSG00000088279"
 [1390] "ENSMUSG00000098989" "ENSMUSG00000064691" "ENSMUSG00000087139"
 [1393] "ENSMUSG00000085324" "ENSMUSG00000020624" "ENSMUSG00000086760"
 [1396] "ENSMUSG00000086135" "ENSMUSG00000078600" "ENSMUSG00000075431"
 [1399] "ENSMUSG00000089460" "ENSMUSG00000084130" "ENSMUSG00000064588"
 [1402] "ENSMUSG00000081318" "ENSMUSG00000078586" "ENSMUSG00000086588"
 [1405] "ENSMUSG00000085336" "ENSMUSG00000110653" "ENSMUSG00000082562"
 [1408] "ENSMUSG00000085059" "ENSMUSG00000084887" "ENSMUSG00000080616"
 [1411] "ENSMUSG00000083155" "ENSMUSG00000099690" "ENSMUSG00000085970"
 [1414] "ENSMUSG00000112240" "ENSMUSG00000092168" "ENSMUSG00000112723"
 [1417] "ENSMUSG00000112559" "ENSMUSG00000088209" "ENSMUSG00000102096"
 [1420] "ENSMUSG00000099360" "ENSMUSG00000077259" "ENSMUSG00000112787"
 [1423

...

 [7522] "ENSMUSG00000092815" "ENSMUSG00000109456" "ENSMUSG00000109082"
 [7525] "ENSMUSG00000110339" "ENSMUSG00000096743" "ENSMUSG00000092314"
 [7528] "ENSMUSG00000096691" "ENSMUSG00000092387" "ENSMUSG00000092173"
 [7531] "ENSMUSG00000091651" "ENSMUSG00000092619" "ENSMUSG00000090482"
 [7534] "ENSMUSG00000096668" "ENSMUSG00000090892" "ENSMUSG00000092614"
 [7537] "ENSMUSG00000092204" "ENSMUSG00000090727" "ENSMUSG00000096707"
 [7540] "ENSMUSG00000092573" "ENSMUSG00000109978" "ENSMUSG00000092611"
 [7543] "ENSMUSG00000092620" "ENSMUSG00000092561" "ENSMUSG00000096413"
 [7546] "ENSMUSG00000092551" "ENSMUSG00000093519" "ENSMUSG00000109661"
 [7549] "ENSMUSG00000090342" "ENSMUSG00000110306" "ENSMUSG00000094107"
 [7552] "ENSMUSG00000094280" "ENSMUSG00000093681" "ENSMUSG00000095914"
 [7555] "ENSMUSG00000110108" "ENSMUSG00000096180" "ENSMUSG00000093487"
 [7558] "ENSMUSG00000110098" "ENSMUSG00000034122" "ENSMUSG00000107520"
 [7561] "ENSMUSG00000046716" "ENSMUSG00000110232" "ENSMUSG00000093595"
 [7564

 [7867] "ENSMUSG00000118562" "ENSMUSG00000108430" "ENSMUSG00000108404"
 [7870] "ENSMUSG00000108302" "ENSMUSG00000108675" "ENSMUSG00000108809"
 [7873] "ENSMUSG00000094866" "ENSMUSG00000054102" "ENSMUSG00000064793"
 [7876] "ENSMUSG00000118574" "ENSMUSG00000060747" "ENSMUSG00000089219"
 [7879] "ENSMUSG00000065385" "ENSMUSG00000065659" "ENSMUSG00000109467"
 [7882] "ENSMUSG00000099536" "ENSMUSG00000099357" "ENSMUSG00000101554"
 [7885] "ENSMUSG00000062556" "ENSMUSG00000036521" "ENSMUSG00000101325"
 [7888] "ENSMUSG00000100358" "ENSMUSG00000101658" "ENSMUSG00000100754"
 [7891] "ENSMUSG00000101065" "ENSMUSG00000099598" "ENSMUSG00000100734"
 [7894] "ENSMUSG00000102002" "ENSMUSG00000100639" "ENSMUSG00000102041"
 [7897] "ENSMUSG00000094053" "ENSMUSG00000078759" "ENSMUSG00000100517"
 [7900] "ENSMUSG00000099961" "ENSMUSG00000099612" "ENSMUSG00000101765"
 [7903] "ENSMUSG00000101270" "ENSMUSG00000101248" "ENSMUSG00000100926"
 [7906] "ENSMUSG00000101039" "ENSMUSG00000101025" "ENSMUSG00000099396"
 [7909

 [8212] "ENSMUSG00000094765" "ENSMUSG00000096281" "ENSMUSG00000094213"
 [8215] "ENSMUSG00000096454" "ENSMUSG00000093870" "ENSMUSG00000093854"
 [8218] "ENSMUSG00000094524" "ENSMUSG00000094099" "ENSMUSG00000094093"
 [8221] "ENSMUSG00000096287" "ENSMUSG00000094912" "ENSMUSG00000094636"
 [8224] "ENSMUSG00000094567" "ENSMUSG00000094458" "ENSMUSG00000095628"
 [8227] "ENSMUSG00000096185" "ENSMUSG00000096447" "ENSMUSG00000096814"
 [8230] "ENSMUSG00000096306" "ENSMUSG00000096067" "ENSMUSG00000094032"
 [8233] "ENSMUSG00000096090" "ENSMUSG00000096595" "ENSMUSG00000095402"
 [8236] "ENSMUSG00000095764" "ENSMUSG00000075968" "ENSMUSG00000095924"
 [8239] "ENSMUSG00000094998" "ENSMUSG00000095357" "ENSMUSG00000095021"
 [8242] "ENSMUSG00000095660" "ENSMUSG00000094037" "ENSMUSG00000094438"
 [8245] "ENSMUSG00000096392" "ENSMUSG00000096821" "ENSMUSG00000095714"
 [8248] "ENSMUSG00000095945" "ENSMUSG00000094847" "ENSMUSG00000094148"
 [8251] "ENSMUSG00000096812" "ENSMUSG00000094129" "ENSMUSG00000095587"
 [8254

 [8557] "ENSMUSG00000080891" "ENSMUSG00000083254" "ENSMUSG00000051362"
 [8560] "ENSMUSG00000084356" "ENSMUSG00000095000" "ENSMUSG00000073951"
 [8563] "ENSMUSG00000073950" "ENSMUSG00000109951" "ENSMUSG00000082739"
 [8566] "ENSMUSG00000059874" "ENSMUSG00000080367" "ENSMUSG00000109659"
 [8569] "ENSMUSG00000073949" "ENSMUSG00000073948" "ENSMUSG00000046396"
 [8572] "ENSMUSG00000045584" "ENSMUSG00000096584" "ENSMUSG00000094119"
 [8575] "ENSMUSG00000078624" "ENSMUSG00000073947" "ENSMUSG00000047544"
 [8578] "ENSMUSG00000073946" "ENSMUSG00000073945" "ENSMUSG00000073944"
 [8581] "ENSMUSG00000081315" "ENSMUSG00000050085" "ENSMUSG00000099687"
 [8584] "ENSMUSG00000073943" "ENSMUSG00000081038" "ENSMUSG00000082077"
 [8587] "ENSMUSG00000050281" "ENSMUSG00000061626" "ENSMUSG00000085700"
 [8590] "ENSMUSG00000058200" "ENSMUSG00000110259" "ENSMUSG00000042219"
 [8593] "ENSMUSG00000109837" "ENSMUSG00000073938" "ENSMUSG00000073937"
 [8596] "ENSMUSG00000080751" "ENSMUSG00000055621" "ENSMUSG00000084236"
 [8599

 [8902] "ENSMUSG00000110463" "ENSMUSG00000074454" "ENSMUSG00000071165"
 [8905] "ENSMUSG00000094406" "ENSMUSG00000074446" "ENSMUSG00000061845"
 [8908] "ENSMUSG00000087663" "ENSMUSG00000094687" "ENSMUSG00000094516"
 [8911] "ENSMUSG00000083509" "ENSMUSG00000074444" "ENSMUSG00000074443"
 [8914] "ENSMUSG00000095431" "ENSMUSG00000079116" "ENSMUSG00000094321"
 [8917] "ENSMUSG00000095328" "ENSMUSG00000080722" "ENSMUSG00000074441"
 [8920] "ENSMUSG00000094180" "ENSMUSG00000074440" "ENSMUSG00000074439"
 [8923] "ENSMUSG00000083251" "ENSMUSG00000082211" "ENSMUSG00000095221"
 [8926] "ENSMUSG00000082918" "ENSMUSG00000096295" "ENSMUSG00000094362"
 [8929] "ENSMUSG00000094662" "ENSMUSG00000084344" "ENSMUSG00000074436"
 [8932] "ENSMUSG00000096558" "ENSMUSG00000079114" "ENSMUSG00000095925"
 [8935] "ENSMUSG00000095066" "ENSMUSG00000075934" "ENSMUSG00000065956"
 [8938] "ENSMUSG00000083987" "ENSMUSG00000074434" "ENSMUSG00000095703"
 [8941] "ENSMUSG00000079113" "ENSMUSG00000060070" "ENSMUSG00000084366"
 [8944

In [41]:
%%R
reactome_ensembl_ids <- intersect(ensembl2rxns.df$V1,ensembl_wo_ids)
print(reactome_ensembl_ids)

 "ENSMUSG00000043131"
[6829] "ENSMUSG00000043140" "ENSMUSG00000043241" "ENSMUSG00000043257"
[6832] "ENSMUSG00000043279" "ENSMUSG00000043300" "ENSMUSG00000043301"
[6835] "ENSMUSG00000043366" "ENSMUSG00000043411" "ENSMUSG00000043415"
[6838] "ENSMUSG00000043421" "ENSMUSG00000043424" "ENSMUSG00000043448"
[6841] "ENSMUSG00000043461" "ENSMUSG00000043463" "ENSMUSG00000043468"
[6844] "ENSMUSG00000043487" "ENSMUSG00000043510" "ENSMUSG00000043556"
[6847] "ENSMUSG00000043557" "ENSMUSG00000043614" "ENSMUSG00000043635"
[6850] "ENSMUSG00000043648" "ENSMUSG00000043659" "ENSMUSG00000043673"
[6853] "ENSMUSG00000043683" "ENSMUSG00000043702" "ENSMUSG00000043705"
[6856] "ENSMUSG00000043716" "ENSMUSG00000043719" "ENSMUSG00000043733"
[6859] "ENSMUSG00000043822" "ENSMUSG00000043866" "ENSMUSG00000043881"
[6862] "ENSMUSG00000043885" "ENSMUSG00000043895" "ENSMUSG00000043909"
[6865] "ENSMUSG00000043911" "ENSMUSG00000043943" "ENSMUSG00000043953"
[6868] "ENSMUSG00000043962" "ENSMUSG00000043969" "ENSMUSG00000043972

[7177] "ENSMUSG00000047953" "ENSMUSG00000047959" "ENSMUSG00000047963"
[7180] "ENSMUSG00000047976" "ENSMUSG00000047989" "ENSMUSG00000048001"
[7183] "ENSMUSG00000048003" "ENSMUSG00000048012" "ENSMUSG00000048013"
[7186] "ENSMUSG00000048029" "ENSMUSG00000048039" "ENSMUSG00000048065"
[7189] "ENSMUSG00000048076" "ENSMUSG00000048087" "ENSMUSG00000048096"
[7192] "ENSMUSG00000048100" "ENSMUSG00000048118" "ENSMUSG00000048120"
[7195] "ENSMUSG00000048126" "ENSMUSG00000048142" "ENSMUSG00000048154"
[7198] "ENSMUSG00000048163" "ENSMUSG00000048175" "ENSMUSG00000048217"
[7201] "ENSMUSG00000048218" "ENSMUSG00000048222" "ENSMUSG00000048232"
[7204] "ENSMUSG00000048240" "ENSMUSG00000048249" "ENSMUSG00000048280"
[7207] "ENSMUSG00000048284" "ENSMUSG00000048304" "ENSMUSG00000048337"
[7210] "ENSMUSG00000048349" "ENSMUSG00000048368" "ENSMUSG00000048371"
[7213] "ENSMUSG00000048373" "ENSMUSG00000048376" "ENSMUSG00000048379"
[7216] "ENSMUSG00000048387" "ENSMUSG00000048402" "ENSMUSG00000048439"
[7219] "ENSMUSG00000

[7528] "ENSMUSG00000052752" "ENSMUSG00000052759" "ENSMUSG00000052763"
[7531] "ENSMUSG00000052783" "ENSMUSG00000052798" "ENSMUSG00000052819"
[7534] "ENSMUSG00000052821" "ENSMUSG00000052833" "ENSMUSG00000052850"
[7537] "ENSMUSG00000052852" "ENSMUSG00000052889" "ENSMUSG00000052911"
[7540] "ENSMUSG00000052914" "ENSMUSG00000052915" "ENSMUSG00000052920"
[7543] "ENSMUSG00000052921" "ENSMUSG00000052922" "ENSMUSG00000052934"
[7546] "ENSMUSG00000052957" "ENSMUSG00000052962" "ENSMUSG00000052974"
[7549] "ENSMUSG00000052997" "ENSMUSG00000053004" "ENSMUSG00000053040"
[7552] "ENSMUSG00000053044" "ENSMUSG00000053062" "ENSMUSG00000053063"
[7555] "ENSMUSG00000053110" "ENSMUSG00000053113" "ENSMUSG00000053119"
[7558] "ENSMUSG00000053137" "ENSMUSG00000053141" "ENSMUSG00000053158"
[7561] "ENSMUSG00000053178" "ENSMUSG00000053198" "ENSMUSG00000053199"
[7564] "ENSMUSG00000053216" "ENSMUSG00000053219" "ENSMUSG00000053279"
[7567] "ENSMUSG00000053291" "ENSMUSG00000053293" "ENSMUSG00000053317"
[7570] "ENSMUSG00000

[7879] "ENSMUSG00000057667" "ENSMUSG00000057672" "ENSMUSG00000057691"
[7882] "ENSMUSG00000057722" "ENSMUSG00000057729" "ENSMUSG00000057738"
[7885] "ENSMUSG00000057751" "ENSMUSG00000057788" "ENSMUSG00000057789"
[7888] "ENSMUSG00000057841" "ENSMUSG00000057842" "ENSMUSG00000057863"
[7891] "ENSMUSG00000057880" "ENSMUSG00000057897" "ENSMUSG00000057914"
[7894] "ENSMUSG00000057933" "ENSMUSG00000057948" "ENSMUSG00000057963"
[7897] "ENSMUSG00000057967" "ENSMUSG00000058076" "ENSMUSG00000058099"
[7900] "ENSMUSG00000058119" "ENSMUSG00000058124" "ENSMUSG00000058135"
[7903] "ENSMUSG00000058145" "ENSMUSG00000058152" "ENSMUSG00000058192"
[7906] "ENSMUSG00000058230" "ENSMUSG00000058239" "ENSMUSG00000058246"
[7909] "ENSMUSG00000058248" "ENSMUSG00000058250" "ENSMUSG00000058254"
[7912] "ENSMUSG00000058258" "ENSMUSG00000058267" "ENSMUSG00000058290"
[7915] "ENSMUSG00000058291" "ENSMUSG00000058301" "ENSMUSG00000058317"
[7918] "ENSMUSG00000058318" "ENSMUSG00000058325" "ENSMUSG00000058331"
[7921] "ENSMUSG00000

[8230] "ENSMUSG00000062519" "ENSMUSG00000062542" "ENSMUSG00000062585"
[8233] "ENSMUSG00000062591" "ENSMUSG00000062609" "ENSMUSG00000062627"
[8236] "ENSMUSG00000062647" "ENSMUSG00000062683" "ENSMUSG00000062694"
[8239] "ENSMUSG00000062727" "ENSMUSG00000062729" "ENSMUSG00000062732"
[8242] "ENSMUSG00000062743" "ENSMUSG00000062751" "ENSMUSG00000062773"
[8245] "ENSMUSG00000062778" "ENSMUSG00000062785" "ENSMUSG00000062794"
[8248] "ENSMUSG00000062797" "ENSMUSG00000062825" "ENSMUSG00000062861"
[8251] "ENSMUSG00000062866" "ENSMUSG00000062867" "ENSMUSG00000062906"
[8254] "ENSMUSG00000062908" "ENSMUSG00000062931" "ENSMUSG00000062937"
[8257] "ENSMUSG00000062944" "ENSMUSG00000062949" "ENSMUSG00000062960"
[8260] "ENSMUSG00000062981" "ENSMUSG00000062991" "ENSMUSG00000062997"
[8263] "ENSMUSG00000063011" "ENSMUSG00000063021" "ENSMUSG00000063047"
[8266] "ENSMUSG00000063049" "ENSMUSG00000063060" "ENSMUSG00000063063"
[8269] "ENSMUSG00000063065" "ENSMUSG00000063077" "ENSMUSG00000063089"
[8272] "ENSMUSG00000

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



[8905] "ENSMUSG00000078253" "ENSMUSG00000078348" "ENSMUSG00000078354"
[8908] "ENSMUSG00000078355" "ENSMUSG00000078427" "ENSMUSG00000078440"
[8911] "ENSMUSG00000078486" "ENSMUSG00000078497" "ENSMUSG00000078502"
[8914] "ENSMUSG00000078578" "ENSMUSG00000078580" "ENSMUSG00000078591"
[8917] "ENSMUSG00000078597" "ENSMUSG00000078619" "ENSMUSG00000078630"
[8920] "ENSMUSG00000078650" "ENSMUSG00000078651" "ENSMUSG00000078652"
[8923] "ENSMUSG00000078656" "ENSMUSG00000078676" "ENSMUSG00000078713"
[8926] "ENSMUSG00000078762" "ENSMUSG00000078765" "ENSMUSG00000078768"
[8929] "ENSMUSG00000078780" "ENSMUSG00000078783" "ENSMUSG00000078789"
[8932] "ENSMUSG00000078798" "ENSMUSG00000078810" "ENSMUSG00000078812"
[8935] "ENSMUSG00000078813" "ENSMUSG00000078815" "ENSMUSG00000078816"
[8938] "ENSMUSG00000078851" "ENSMUSG00000078861" "ENSMUSG00000078866"
[8941] "ENSMUSG00000078868" "ENSMUSG00000078906" "ENSMUSG00000078908"
[8944] "ENSMUSG00000078919" "ENSMUSG00000078923" "ENSMUSG00000078937"
[8947] "ENSMUSG00000

[9256] "ENSMUSG00000110061" "ENSMUSG00000110091" "ENSMUSG00000110097"
[9259] "ENSMUSG00000110195" "ENSMUSG00000110206" "ENSMUSG00000110576"
[9262] "ENSMUSG00000110949" "ENSMUSG00000111259" "ENSMUSG00000111709"
[9265] "ENSMUSG00000111915" "ENSMUSG00000112160" "ENSMUSG00000112170"
[9268] "ENSMUSG00000112449" "ENSMUSG00000112640" "ENSMUSG00000113262"
[9271] "ENSMUSG00000113902" "ENSMUSG00000114279" "ENSMUSG00000114456"
[9274] "ENSMUSG00000114515" "ENSMUSG00000114755" "ENSMUSG00000114797"
[9277] "ENSMUSG00000115067" "ENSMUSG00000115293" "ENSMUSG00000115338"
[9280] "ENSMUSG00000115958" "ENSMUSG00000115987" "ENSMUSG00000116048"
[9283] "ENSMUSG00000116158" "ENSMUSG00000116207" "ENSMUSG00000116336"
[9286] "ENSMUSG00000116564" "ENSMUSG00000116780" "ENSMUSG00000116876"
[9289] "ENSMUSG00000116988" "ENSMUSG00000117098" "ENSMUSG00000117406"
[9292] "ENSMUSG00000117621" "ENSMUSG00000117789" "ENSMUSG00000118193"
[9295] "ENSMUSG00000118346" "ENSMUSG00000118462" "ENSMUSG00000118541"


In [42]:
%%R
saveRDS(reactome_ensembl_ids,file=paste(OUT_DIR,"reactome_ensembl_ids10vs0.Rds",sep=""))

In [43]:
%%R
tcdd.df <- tcdd.df[reactome_ensembl_ids,]
print(tcdd.df)

                   SRR7817611 SRR7817612 SRR7817614 SRR7817615 SRR7817616
ENSMUSG00000000001      54414      55963      71636      56315      55499
ENSMUSG00000000028        483        371        785        445        508
ENSMUSG00000000049     867859     869565    1380723    1012617    1016838
ENSMUSG00000000058       1200       1446       1997       1048       1967
ENSMUSG00000000085       5970       5071       7537       5648       4328
ENSMUSG00000000088     102144     104159     135652      84658      82419
ENSMUSG00000000120       3593       3842       5318       2860       2708
ENSMUSG00000000126        299        599        300        649        500
ENSMUSG00000000127       3431       2944       5020       3461       4270
ENSMUSG00000000142       3978       3978       5202       6884       5789
ENSMUSG00000000149      63577      67422      87839      70522      62028
ENSMUSG00000000154      46678      50051      69606      53885      54402
ENSMUSG00000000168      30504      315

ENSMUSG00000001105      23051      23508      24998      19259      18570
ENSMUSG00000001119       3665       2330       3866       3164       4668
ENSMUSG00000001123     264484     265650     387854     298801     304169
ENSMUSG00000001127      39275      40798      60205      46522      49501
ENSMUSG00000001128      11737      12898      11458      10088      10022
ENSMUSG00000001131         50         50          0          0        243
ENSMUSG00000001143       8646       7832      13080       9712       8308
ENSMUSG00000001155     353045     344441     499448     338014     330591
ENSMUSG00000001158       5949       5270       9361       5246       5195
ENSMUSG00000001173       3032       3171       6074       3164       4350
ENSMUSG00000001175     133974     138059     191807     151772     155612
ENSMUSG00000001211     111979     113748     151358     114714     112058
ENSMUSG00000001225          0         49          0          0          0
ENSMUSG00000001228        350        5

ENSMUSG00000002249       6461       6663       9627       7581       6350
ENSMUSG00000002250      27687      30336      30212      33623      35205
ENSMUSG00000002257        200        146        230        350        300
ENSMUSG00000002279      16221      20624      27187      22749      21433
ENSMUSG00000002289      12629      11667      31828      11101      12772
ENSMUSG00000002297       1260       1519       1381        945        788
ENSMUSG00000002307       7276       7464      10207       8225       8229
ENSMUSG00000002308       2714       2680       2652       3314       2534
ENSMUSG00000002324       1077        788       2328        448        648
ENSMUSG00000002325      20146      21517      27420      24704      26582
ENSMUSG00000002326       8550       8826      10024       7821       8404
ENSMUSG00000002346      64896      67271      82118      63564      64902
ENSMUSG00000002365      20649      21494      33607      24690      25591
ENSMUSG00000002379      27330      247

ENSMUSG00000003379        300        444        100        300        150
ENSMUSG00000003402      66313      63844      90087      66382      62230
ENSMUSG00000003411        200        393        200        350        250
ENSMUSG00000003418          0         50        100          0         50
ENSMUSG00000003421      12362      14159      19224      13609      12810
ENSMUSG00000003429      32654      33682      37756      23635      24943
ENSMUSG00000003435      46892      46870      45511      42232      41638
ENSMUSG00000003436          0          0          0          0          0
ENSMUSG00000003437      18312      16939      24629      16270      13970
ENSMUSG00000003444       2291       1597       3399       2106       2839
ENSMUSG00000003452        100         99          0          0        100
ENSMUSG00000003458      23564      23364      28610      27214      26430
ENSMUSG00000003464      55268      60757      80567      62438      60499
ENSMUSG00000003476        150        3

ENSMUSG00000004654          0          0          0          0          0
ENSMUSG00000004655      41227      41735      48111      41064      41048
ENSMUSG00000004665       3536       2943       3696       3349       4333
ENSMUSG00000004667      16817      18820      20609      14270      15317
ENSMUSG00000004668          0         50          0          0          0
ENSMUSG00000004677      20901      21276      22605      18446      17750
ENSMUSG00000004709         50        200        100         50        240
ENSMUSG00000004730      11537      12030       5172       8580       8435
ENSMUSG00000004768       5440       4544       7419       5263       5385
ENSMUSG00000004771      31430      34095      43689      31829      30566
ENSMUSG00000004788      11441      12893      14088      10703      10919
ENSMUSG00000004789     124044     127284     159502     122528     127971
ENSMUSG00000004791         50         50          0          0          0
ENSMUSG00000004815      23720      232

...

ENSMUSG00000014551      20598      21506      22197      26822      27648
ENSMUSG00000014554       5317       5134       4351       5943       6759
ENSMUSG00000014599       5699       3143       4045       5136       6226
ENSMUSG00000014602         50         95          0        297        100
ENSMUSG00000014606      24269      25246      23161      28504      28243
ENSMUSG00000014609        296        248        150         50          0
ENSMUSG00000014704        250        250         50        150        250
ENSMUSG00000014763      12378      13247      12332      13736      14966
ENSMUSG00000014767      10132      11986      12584      13677      13808
ENSMUSG00000014769      56560      47930      49155      62131      57270
ENSMUSG00000014773       1103       1633       1247       1073       1240
ENSMUSG00000014786        349        380        131        278         91
ENSMUSG00000014850       7263       9186       8572       9189       8436
ENSMUSG00000014852       2997       25

ENSMUSG00000016252      64532      67297      69334      60178      61057
ENSMUSG00000016253      14304      12698      11876      18178      18382
ENSMUSG00000016255         87          0        150         43        150
ENSMUSG00000016256      99480      72400      70159     100052     102438
ENSMUSG00000016283        150          0        100         50          0
ENSMUSG00000016308       9319      10120      10077      14140      16797
ENSMUSG00000016319     141197     138091     141119     174555     173121
ENSMUSG00000016327          0          0          0          0          0
ENSMUSG00000016356        791        740        530        652        311
ENSMUSG00000016409       5251       5419       5986       6133       6499
ENSMUSG00000016427      26327      29045      30433      25346      25947
ENSMUSG00000016477       3424       5183       3902       4607       5680
ENSMUSG00000016487      16272      16209      17172      22716      19796
ENSMUSG00000016493       5200       38

ENSMUSG00000018143       7054       5572       6978       7391       7161
ENSMUSG00000018160      19297      21134      18682      26488      25491
ENSMUSG00000018166      96528      82301      80534     114719     115393
ENSMUSG00000018167       7980       5646       6298      10069       9507
ENSMUSG00000018169        449        584        500        590        694
ENSMUSG00000018189       9435       8462       9004       9630       9057
ENSMUSG00000018209      13051      11080      10123      15372      15513
ENSMUSG00000018263          0          0          0          0          0
ENSMUSG00000018286      55234      49048      48452      57011      53384
ENSMUSG00000018293      37920      32981      32202      42935      39122
ENSMUSG00000018326      41095      41052      42466      46703      47746
ENSMUSG00000018334       1397       1074        866       1028        736
ENSMUSG00000018339       2230       1968       1637       2695       3101
ENSMUSG00000018340      56360      558

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



      26855      16656      18043
ENSMUSG00000002210       6588       7570       7671       5511       6366
ENSMUSG00000002221      14877      20089      19763      14099      14251
ENSMUSG00000002227      14431      16060      15544      10192      10607
ENSMUSG00000002233       8851      12177      10469       6107       5532
ENSMUSG00000002249       6329       8200       7762       5539       5602
ENSMUSG00000002250       6161      13103      14023       6210       6951
ENSMUSG00000002257        399        833        543        442        443
ENSMUSG00000002279      30834      38526      39819      21687      21494
ENSMUSG00000002289     135062     137337     140416     102021     107647
ENSMUSG00000002297       3314      12451      13222       2646       2819
ENSMUSG00000002307      10413      15644      15277       9323       8233
ENSMUSG00000002308       3407       4375       4715       2126       1571
ENSMUSG00000002324       2776       3072       2446       2164       1727
ENSM

In [44]:
%%R
saveRDS(tcdd.df,file=paste(OUT_DIR,"tcdd_df10vs0_time_cross.Rds",sep=""))

In [45]:
%%R
saveRDS(deleted_data, file=paste(OUT_DIR, "format_name_deleted_data10vs0_time_cross.Rds", sep=""))